<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-opensource/blob/main/TAI_RAG_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Together AI + RAG

[Together AI](https://python.langchain.com/docs/integrations/llms/together) has a broad set of OSS LLMs via inference API.

See [here](https://docs.together.ai/docs/inference-models). We use `"mistralai/Mixtral-8x7B-Instruct-v0.1` for RAG on the Mixtral paper.

Download the paper:
https://arxiv.org/pdf/2401.04088.pdf

In [2]:
!wget -q https://arxiv.org/pdf/2401.04088.pdf

In [11]:
! pip install --quiet pypdf chromadb tiktoken openai langchain-together langchain-community sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 28.4 MB/s eta 0:00:00


In [16]:
import os
os.environ['TOGETHER_API_KEY'] = '4faeef5f54789f54e13a6e05a6e413be38ba3b30c2059c6a8c9b955a413ce99b'

In [6]:
# Load
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/2401.04088.pdf")
data = loader.load()
print(len(data))

13


In [7]:
print(data[0])

page_content='Mixtral of Experts
Albert Q. Jiang, Alexandre Sablayrolles, Antoine Roux, Arthur Mensch,
Blanche Savary, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas,
Emma Bou Hanna, Florian Bressand, Gianna Lengyel, Guillaume Bour,
Guillaume Lample, Lélio Renard Lavaud, Lucile Saulnier, Marie-Anne Lachaux,
Pierre Stock, Sandeep Subramanian, Sophia Yang, Szymon Antoniak, Teven Le Scao,
Théophile Gervet, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed
Abstract
We introduce Mixtral 8x7B, a Sparse Mixture of Experts (SMoE) language
model. Mixtral has the same architecture as Mistral 7B, with the difference
that each layer is composed of 8 feedforward blocks (i.e. experts). For every
token, at each layer, a router network selects two experts to process the current
state and combine their outputs. Even though each token only sees two experts,
the selected experts can be different at each timestep. As a result, each token
has access to 47B parameters, but only uses

In [8]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
print(len(all_splits))

23


In [12]:
# loading embedding model from Hugging Face
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding_model_name = "BAAI/bge-large-en-v1.5"
embeddings = HuggingFaceBgeEmbeddings(model_name=embedding_model_name,)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
# Add to vectorDB
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

"""
from langchain_together.embeddings import TogetherEmbeddings
embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval")
"""
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)

retriever = vectorstore.as_retriever()

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# RAG prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

prompt



ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [17]:
# LLM
from langchain_together import Together

llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.0,
    max_tokens=2000,
    top_k=1,
)



In [18]:
# RAG chain
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
chain.invoke("What are the Architectural details of Mixtral?")

'\nAnswer: The architectural details of Mixtral are as follows:\n\n* The model is based on a transformer architecture and uses the same modifications as described in [18].\n* Mixtral supports a fully dense context length of 32k tokens.\n* The feedforward block picks from a set of 8 distinct groups of parameters.\n* At every layer, for every token, a router network chooses two of these groups (the “experts”) to process the token and combine their output additively.\n* The model has a dimension of 4096, 32 layers, head dimension of 128, hidden dimension of 14336, 32 number of heads, 8 number of kv heads, context length of 32768, vocabulary size of 32000, 8 number of experts, and 2 top k experts.\n\nThese details are provided in Table 1 of the document.'

# Thank You